In [48]:
import numpy as np

def B_t(ts, d, B):
    temp = np.power(ts,-d)
    return np.sum(temp) + B

def activation(t,ts,w,s,d,B):
    return B_t(t+ts,d,B) +  np.sum(w.reshape(1,-1)*s)


ts = np.array([4,3,2,1])
w = np.array([1,1])
s = np.array([[1.5,0.0],
              [0.0,1.5]])

d = .5
B = 0
t = 4

activation(t,ts,w,s,d,B)


4.586979749566321

In [2]:
import pandas as pd
import numpy as np
import sys
df = pd.read_csv("../../theo_prediction_repo/data/assistments17/preprocessed_data.csv",delimiter="\t")
#df = df[df['user_id'] < 20]
now = np.max(df["timestamp"])
print(now)

def dws_to_ints(day_windows):
    day_windows = np.array(day_windows)
    
def calc_features(df,windows=[1.0/24,1,7,30]):
    for i in range(len(windows)+1):
        df['lop_'+str(i)] = 0
        df['ltc_'+str(i)] = 0
        
    windows = [0.0] + [86400.0*x for x in windows] + [float("inf")]
    gb = df.groupby(["user_id","skill_id"])
    for (u,s),g in gb:
        #print(g)
        np_ts = np.array(g['timestamp'])
        np_corr = np.array(g['correct'])
        diffs = np_ts.reshape(-1,1) - np_ts.reshape(1,-1)
        for i in range(len(windows)-1):
            upper = windows[i+1]
            lower = windows[i]
            in_win = np.logical_and(diffs > lower,diffs <= upper)
            c_in_win = in_win * np_corr.reshape(1,-1)
            df['lop_'+str(i)].iloc[g.index] = np.log(1+in_win.sum(axis=1))
            df['ltc_'+str(i)].iloc[g.index] = np.log(1+c_in_win.sum(axis=1))
         
    return df
df = calc_features(df)

84797322


/home/danny/.local/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
print(df)
df.to_csv("a17_tw_windows.txt",sep="\t")

        user_id  item_id  timestamp  correct  skill_id     lop_0     ltc_0  \
0             0     1117    1049253        0       339  0.000000  0.000000   
1             0     1118    1049302        1       339  0.693147  0.000000   
2             0     1119    1049306        0       338  0.000000  0.000000   
3             0     1119    1049312        0       338  0.693147  0.000000   
4             0     1120    1049330        1        44  0.000000  0.000000   
...         ...      ...        ...      ...       ...       ...       ...   
934633     1707     1711   52026416        1       158  2.890372  1.386294   
934634     1707     1712   52026420        0       158  2.944439  1.609438   
934635     1707     1712   52026491        0       158  2.995732  1.609438   
934636     1707     1712   52026495        1       158  3.044522  1.609438   
934637     1707      835   52026498        1       174  0.000000  0.000000   

        lop_1  ltc_1  lop_2  ltc_2  lop_3  ltc_3  lop_4  ltc_4 

In [53]:
print(df)
import pandas as pd
n_skills = df["skill_id"].max()+1
n_students = df["user_id"].max()+1

Y = df["correct"].to_numpy(dtype=np.float32)
#[]'user_id','item_id',
X = df.loc[:,~df.columns.isin(['item_id','timestamp','correct'])].to_numpy(dtype=np.float32)
print(Y.shape,X.shape)

        user_id  item_id  timestamp  correct  skill_id  op_0  tc_0  op_1  \
0             0     1117    1049253        0       339     0     0     0   
1             0     1118    1049302        1       339     1     0     0   
2             0     1119    1049306        0       338     0     0     0   
3             0     1119    1049312        0       338     1     0     0   
4             0     1120    1049330        1        44     0     0     0   
...         ...      ...        ...      ...       ...   ...   ...   ...   
934633     1707     1711   52026416        1       158    17     3     0   
934634     1707     1712   52026420        0       158    18     4     0   
934635     1707     1712   52026491        0       158    19     4     0   
934636     1707     1712   52026495        1       158    20     4     0   
934637     1707      835   52026498        1       174     0     0     0   

        tc_1  op_2  tc_2  op_3  tc_3  op_4  tc_4  
0          0     0     0     0     0

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class DASH(torch.nn.Module):
    def __init__(self,n_students, n_kcs, windows=[1/24,1,7,30],add_inf=True):
        super(DASH, self).__init__()
        self.windows = windows
        self.n_wins = len(windows)+1 if(add_inf) else len(windows)
        self.phi = nn.Parameter(torch.zeros(self.n_wins))
        self.psi = nn.Parameter(torch.zeros(self.n_wins))
        self.alpha = nn.Parameter(torch.zeros(n_students))
        self.delta = nn.Parameter(torch.zeros(n_kcs))

    def forward(self, x):
        s = 0
        for i,w in enumerate(self.windows):
            op = x[2+(i)*2]
            tc = x[2+(i)*2+1]
            s += self.phi[i]*torch.log(1+tc) - self.psi[i]*torch.log(1+op)
        
        return F.sigmoid(self.alpha[x[0].int()]-self.delta[x[1].int()],s) # self.alpha[x[0].int()]-self.delta[x[1].int()]

In [62]:
import torch.optim as optim
print(n_skills)
net = DASH(n_students,n_skills)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
X = torch.from_numpy(X) if isinstance(X,np.ndarray) else X
Y = torch.from_numpy(Y) if isinstance(Y,np.ndarray) else Y

for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    #for i, data in enumerate(trainloader, 0):
    for i in range(len(Y)):
        # get the inputs; data is a list of [inputs, labels]
        
        y,x = Y[i],X[i]

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(x)
        #print(outputs,y)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10000 == 9999:    # print every 10000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0

print('Finished Training')

411
[1, 10000] loss: 0.671
[1, 20000] loss: 0.667
[1, 30000] loss: 0.643
[1, 40000] loss: 0.665
[1, 50000] loss: 0.658
[1, 60000] loss: 0.648
[1, 70000] loss: 0.666
[1, 80000] loss: 0.641
[1, 90000] loss: 0.638
[1, 100000] loss: 0.651
[1, 110000] loss: 0.647
[1, 120000] loss: 0.648
[1, 130000] loss: 0.646
[1, 140000] loss: 0.656
[1, 150000] loss: 0.658
[1, 160000] loss: 0.657
[1, 170000] loss: 0.651
[1, 180000] loss: 0.651
[1, 190000] loss: 0.642
[1, 200000] loss: 0.665
[1, 210000] loss: 0.643
[1, 220000] loss: 0.639
[1, 230000] loss: 0.651
[1, 240000] loss: 0.658
[1, 250000] loss: 0.654
[1, 260000] loss: 0.648
[1, 270000] loss: 0.648
[1, 280000] loss: 0.662
[1, 290000] loss: 0.651
[1, 300000] loss: 0.657
[1, 310000] loss: 0.652
[1, 320000] loss: 0.644
[1, 330000] loss: 0.634
[1, 340000] loss: 0.651
[1, 350000] loss: 0.654
[1, 360000] loss: 0.668
[1, 370000] loss: 0.664
[1, 380000] loss: 0.647
[1, 390000] loss: 0.667
[1, 400000] loss: 0.660
[1, 410000] loss: 0.664
[1, 420000] loss: 0.6

[4, 650000] loss: 0.651
[4, 660000] loss: 0.654
[4, 670000] loss: 0.654
[4, 680000] loss: 0.643
[4, 690000] loss: 0.643
[4, 700000] loss: 0.653
[4, 710000] loss: 0.649
[4, 720000] loss: 0.654
[4, 730000] loss: 0.660
[4, 740000] loss: 0.659
[4, 750000] loss: 0.641
[4, 760000] loss: 0.654
[4, 770000] loss: 0.654
[4, 780000] loss: 0.663
[4, 790000] loss: 0.663
[4, 800000] loss: 0.666
[4, 810000] loss: 0.659
[4, 820000] loss: 0.642
[4, 830000] loss: 0.655
[4, 840000] loss: 0.656
[4, 850000] loss: 0.658
[4, 860000] loss: 0.664
[4, 870000] loss: 0.654
[4, 880000] loss: 0.648
[4, 890000] loss: 0.649
[4, 900000] loss: 0.668
[4, 910000] loss: 0.652
[4, 920000] loss: 0.648
[4, 930000] loss: 0.660
[5, 10000] loss: 0.668
[5, 20000] loss: 0.666
[5, 30000] loss: 0.643
[5, 40000] loss: 0.665
[5, 50000] loss: 0.658
[5, 60000] loss: 0.648
[5, 70000] loss: 0.666
[5, 80000] loss: 0.641
[5, 90000] loss: 0.638
[5, 100000] loss: 0.651
[5, 110000] loss: 0.646
[5, 120000] loss: 0.648
[5, 130000] loss: 0.646
[

KeyboardInterrupt: 

In [57]:
for name, param in net.named_parameters():
    if param.requires_grad:
        print(name, param.data)
#print(net.parameters())

phi tensor([-0.1015,  0.1419,  0.1439,  0.1933,  0.0000])
psi tensor([-0.1941,  0.1973,  0.0245,  0.0347,  0.0000])
alpha tensor([ 0.1007,  0.2384, -0.5528,  ..., -0.6654, -0.5438,  0.1393])
delta tensor([ 9.9189e-01,  7.9419e-01,  2.2436e-02, -1.7705e-01,  6.9785e-02,
         4.8587e-01, -1.3249e-01,  5.5750e-01,  1.2272e+00,  5.9690e-01,
         4.1889e-01,  8.9203e-01,  1.6421e-02, -8.9410e-02,  3.9388e-01,
         1.0031e+00,  4.5097e-01, -2.8075e-01,  1.9283e-01, -4.8321e-01,
        -4.1701e-01, -1.5378e-01, -4.4443e-02, -2.0668e-01, -1.5037e-01,
        -7.0495e-01,  4.7350e-01,  3.0021e-01,  3.2610e-01,  5.8234e-01,
         1.3569e-01,  5.6145e-01,  6.5499e-03,  2.0457e-01,  2.5157e-01,
         4.8568e-01,  4.6275e-01,  1.8024e-01,  6.0212e-01,  1.2014e+00,
         9.5882e-01, -2.5901e-02,  3.2690e-01,  4.7374e-01,  2.2910e-01,
         5.9374e-01,  3.0172e-01,  2.5434e-01,  4.9535e-01,  3.4592e-01,
        -1.0937e-01,  9.0618e-01,  9.8810e-01,  4.7392e-01,  1.0697e+00,


In [ ]:
from platform import python_version

print(python_version())

import sys
print(sys.executable)
raise ValueError()

In [75]:
def gen_learner_history(now,times,correct,alpha=0.0,delta=0.0,windows=[1/24,1,7,30]):
    times = np.array(times)
    correct = np.array(correct)
    elapse = now-times
    windows = [0.0] + [86400.0*x for x in windows] + [float('inf')]
    lh = {}
    for i in range(len(windows)-1):
        upper = windows[i+1]
        lower = windows[i]
        in_win = np.logical_and(elapse > lower,elapse <= upper)
        c_in_win = in_win * correct
        
        lh['n_w'+str(i)] = in_win.sum()
        lh['c_w'+str(i)] = c_in_win.sum()
    lh["alpha"] = alpha
    lh["delta"] = delta
    return lh

def day(x):
    return x * 86400.0
def hour(x):
    return x * 3600



In [80]:
#---Fit with torch---
#phi tensor([-0.1015,  0.1419,  0.1439,  0.1933,  0.0000])
#psi tensor([-0.1941,  0.1973,  0.0245,  0.0347,  0.0000])

#---Fit with glmer (in R)----
#    ltc_0      lop_0      ltc_1      lop_1      ltc_2      lop_2      ltc_3      lop_3      ltc_4  lop_4  
# 0.038408  -0.011849   0.048038  -0.032408   0.033345  -0.024711  -0.003275   0.005272  -0.043829  0.033895  
    
 
window_profiles = [{
    'scale' : 1/24,
    'phi' : .038408,
    'psi' : -0.011849
},
{
    'scale' : 1,
    'phi' : 0.048038,
    'psi' : -0.032408
},
{
    'scale' : 7,
    'phi' : 0.033345,
    'psi' : -0.024711
},
{
    'scale' : 30,
    'phi' : -0.003275,
    'psi' : 0.005272
},
]

def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

def DASH_activation(lh,wps):
    s = 0
    for i,wp in enumerate(wps):
        s += wp["phi"]*np.log(1+lh["c_w"+str(i)])-wp["psi"]*np.log(1+lh["n_w"+str(i)])
    return lh['alpha'] - lh['delta'] + s

def DASH(lh,wps):
    x = DASH_activation(lh,wps)
    return 1/(1 + np.exp(-x)) 


In [81]:
learner_history = gen_learner_history(day(2)+hour(1),[
        day(0)+hour(0)+0,
        day(0)+hour(0)+30,
        day(0)+hour(0)+40,
        day(1)+hour(3)+0,
        day(1)+hour(3)+30,
        day(1)+hour(3)+40,
        ],[0,0,1,0,1,1],alpha=.05,delta=.3)
print(learner_history)
print(DASH(learner_history,window_profiles))

{'n_w0': 0, 'c_w0': 0, 'n_w1': 3, 'c_w1': 2, 'n_w2': 3, 'c_w2': 1, 'n_w3': 0, 'c_w3': 0, 'n_w4': 0, 'c_w4': 0, 'alpha': 0.05, 'delta': 0.3}
0.4762857747894385


In [ ]:
import numpy as np
from scipy.sparse import load_npz, csr_matrix
data = load_npz("../theo_prediction_repo/data/spanish/X-isscwatw.npz")

In [ ]:
data.shape